Задание

https://www.kaggle.com/competitions/hotel-booking-demand-3

In [ ]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import opendatasets as od

od.download("https://www.kaggle.com/competitions/hotel-booking-demand-3")

100%|██████████| 1.29M/1.29M [00:00<00:00, 99.5MB/s]


Extracting archive ./hotel-booking-demand-3/hotel-booking-demand-3.zip to ./hotel-booking-demand-3


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [ ]:
data = pd.read_csv("hotel-booking-demand-3/train_final.csv")
data.head(5)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,reserved_room_type,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status_date
0,Resort Hotel,0,3,2016,February,7,12,0,2,2,...,E,F,0,No Deposit,0,Transient,81.0,1,0,2016-02-14
1,City Hotel,1,50,2016,November,47,18,1,2,2,...,A,A,0,No Deposit,0,Transient,93.6,0,2,2016-10-09
2,City Hotel,0,14,2015,September,37,7,1,5,2,...,F,F,0,No Deposit,0,Contract,166.0,0,3,2015-09-13
3,City Hotel,0,18,2015,December,50,6,2,0,2,...,A,A,0,No Deposit,0,Transient,107.0,0,0,2015-12-08
4,City Hotel,1,263,2016,September,40,29,2,4,2,...,A,A,0,No Deposit,0,Transient,100.3,0,0,2016-01-18


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44638 entries, 0 to 44637
Data columns (total 29 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                           44638 non-null  object 
 1   is_canceled                     44638 non-null  int64  
 2   lead_time                       44638 non-null  int64  
 3   arrival_date_year               44638 non-null  int64  
 4   arrival_date_month              44638 non-null  object 
 5   arrival_date_week_number        44638 non-null  int64  
 6   arrival_date_day_of_month       44638 non-null  int64  
 7   stays_in_weekend_nights         44638 non-null  int64  
 8   stays_in_week_nights            44638 non-null  int64  
 9   adults                          44638 non-null  int64  
 10  children                        44638 non-null  float64
 11  babies                          44638 non-null  int64  
 12  meal                            

In [ ]:
data.isna().sum()

hotel                             0
is_canceled                       0
lead_time                         0
arrival_date_year                 0
arrival_date_month                0
arrival_date_week_number          0
arrival_date_day_of_month         0
stays_in_weekend_nights           0
stays_in_week_nights              0
adults                            0
children                          0
babies                            0
meal                              0
country                           0
market_segment                    0
distribution_channel              0
is_repeated_guest                 0
previous_cancellations            0
previous_bookings_not_canceled    0
reserved_room_type                0
assigned_room_type                0
booking_changes                   0
deposit_type                      0
days_in_waiting_list              0
customer_type                     0
adr                               0
required_car_parking_spaces       0
total_of_special_requests   

In [ ]:
encoder_hotel = LabelEncoder()
encoder_arrival = LabelEncoder()
encoder_meal = LabelEncoder()
encoder_market = LabelEncoder()
encoder_channel = LabelEncoder()
encoder_reversed = LabelEncoder()
encoder_assigned = LabelEncoder()
encoder_deposit = LabelEncoder()
encoder_type = LabelEncoder()

data['hotel'] = encoder_hotel.fit_transform(data['hotel'])
data['arrival_date_month'] = encoder_arrival.fit_transform(data['arrival_date_month'])
data['meal'] = encoder_meal.fit_transform(data['meal'])
data['market_segment'] = encoder_market.fit_transform(data['market_segment'])
data['distribution_channel'] = encoder_channel.fit_transform(data['distribution_channel'])
data['reserved_room_type'] = encoder_reversed.fit_transform(data['reserved_room_type'])
data['assigned_room_type'] = encoder_assigned.fit_transform(data['assigned_room_type'])
data['deposit_type'] = encoder_deposit.fit_transform(data['deposit_type'])
data['customer_type'] = encoder_type.fit_transform(data['customer_type'])

In [ ]:
data.drop(['country'], axis=1, inplace=True)
data.drop(['reservation_status_date'], axis=1, inplace=True)
data.drop(['arrival_date_year'], axis=1, inplace=True)

In [ ]:
data.head(5)

,hotel,is_canceled,lead_time,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests
0,1,0,3,3,7,12,0,2,2,0.0,...,0,4,5,0,0,0,2,81.0,1,0
1,0,1,50,9,47,18,1,2,2,0.0,...,0,0,0,0,0,0,2,93.6,0,2
2,0,0,14,11,37,7,1,5,2,2.0,...,0,5,5,0,0,0,0,166.0,0,3
3,0,0,18,2,50,6,2,0,2,0.0,...,0,0,0,0,0,0,2,107.0,0,0
4,0,1,263,11,40,29,2,4,2,0.0,...,0,0,0,0,0,0,2,100.3,0,0


In [ ]:
std_lead = StandardScaler()
data_std = data
column_std = std_lead.fit_transform(data[['lead_time']])
data_std['lead_time'] = column_std
std_adr = StandardScaler()
column_std = std_adr.fit_transform(data[['adr']])
data_std['adr'] = column_std

data_std.head(5)

,hotel,is_canceled,lead_time,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests
0,1,0,-0.922579,3,7,12,0,2,2,0.0,...,0,4,5,0,0,0,2,-0.425871,1,0
1,0,1,-0.472752,9,47,18,1,2,2,0.0,...,0,0,0,0,0,0,2,-0.165699,0,2
2,0,0,-0.817301,11,37,7,1,5,2,2.0,...,0,5,5,0,0,0,0,1.329261,0,3
3,0,0,-0.779017,2,50,6,2,0,2,0.0,...,0,0,0,0,0,0,2,0.110993,0,0
4,0,1,1.565827,11,40,29,2,4,2,0.0,...,0,0,0,0,0,0,2,-0.027353,0,0


In [ ]:
data['is_canceled'].value_counts()

0    31354
1    13284
Name: is_canceled, dtype: int64

In [ ]:
X = data_std.drop(['is_canceled'], axis=1)
Y = data_std['is_canceled']

In [ ]:
os = SMOTE(random_state=0, k_neighbors=10)

columns = X.columns

x, y = os.fit_resample(X, Y)
x = pd.DataFrame(x, columns = columns)
y = pd.DataFrame(y, columns = ['is_canceled'])

In [ ]:
y.value_counts()

is_canceled
0              31354
1              31354
dtype: int64

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15)

In [ ]:
scores = []
for i in range(1,15):
    score = {}
    knn = KNeighborsClassifier(i)
    knn.fit(x_train,y_train)
    answers = knn.predict(x_test)
    
    score['k'] = i
    score['accuracy'] = accuracy_score(answers,y_test)
    score['recall'] = recall_score(answers,y_test)
    score['presicion'] = precision_score(answers,y_test)
    
    scores.append(score)

scores = pd.DataFrame(scores)
scores

/usr/local/lib/python3.9/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.9/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.9/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.9/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for exam

,k,accuracy,recall,presicion
0,1,0.848623,0.810335,0.907221
1,2,0.845753,0.869825,0.810371
2,3,0.831083,0.789940,0.898436
3,4,0.835548,0.836933,0.830298
4,5,0.815669,0.775296,0.884937
5,6,0.822579,0.812032,0.835869
6,7,0.805464,0.765587,0.876152
7,8,0.813543,0.797427,0.836726
8,9,0.798235,0.759513,0.868224
9,10,0.805889,0.788075,0.832655


In [ ]:
test = pd.read_csv("hotel-booking-demand-3/test_final.csv")

In [ ]:
test.head(5)

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,reserved_room_type,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status_date
0,City Hotel,73,2016,July,28,6,0,2,1,0.0,...,A,A,0,No Deposit,0,Transient,107.10,0,0,2016-06-12
1,City Hotel,37,2015,October,43,24,2,5,1,1.0,...,A,A,0,No Deposit,0,Transient,87.78,0,0,2015-10-31
2,City Hotel,190,2017,April,14,6,2,3,2,0.0,...,A,A,0,No Deposit,0,Transient,88.40,0,0,2016-09-29
3,City Hotel,287,2016,August,35,24,1,4,2,0.0,...,B,B,1,No Deposit,0,Transient,76.71,0,0,2016-08-29
4,Resort Hotel,386,2016,October,43,20,1,3,2,0.0,...,A,A,0,No Deposit,0,Transient-Party,49.00,0,0,2016-09-14


In [ ]:
test.drop(['country'], axis=1, inplace=True)
test.drop(['reservation_status_date'], axis=1, inplace=True)
test.drop(['arrival_date_year'], axis=1, inplace=True)

In [ ]:
test['hotel'] = encoder_hotel.transform(test['hotel'])
test['arrival_date_month'] = encoder_arrival.transform(test['arrival_date_month'])
test['meal'] = encoder_meal.transform(test['meal'])
test['market_segment'] = encoder_market.transform(test['market_segment'])
test['distribution_channel'] = encoder_channel.transform(test['distribution_channel'])
test['reserved_room_type'] = encoder_reversed.fit_transform(test['reserved_room_type'])
test['assigned_room_type'] = encoder_assigned.fit_transform(test['assigned_room_type'])
test['deposit_type'] = encoder_deposit.transform(test['deposit_type'])
test['customer_type'] = encoder_type.transform(test['customer_type'])

In [ ]:
test_std = test
column_std = std_lead.transform(test[['lead_time']])
test_std['lead_time'] = column_std
column_std = std_adr.transform(test[['adr']])
test_std['adr'] = column_std

In [ ]:
knn = KNeighborsClassifier(1)
knn.fit(x_train,y_train)
answers = knn.predict(test_std)

/usr/local/lib/python3.9/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [ ]:
df = pd.DataFrame(answers, columns=['is_canceled'])
df = df.reset_index()

In [ ]:
df.to_csv("answers_hotel.csv", index=False)